In [1]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

# Principal Component Analysis Methods

### Build a simple curve for risking and pricing

In [2]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2024, 1, 1): Dual(1, {"v1": 1}),
    datetime(2027, 1, 1): Dual(1, {"v2": 1}),
    datetime(2032, 1, 1): Dual(1, {"v3": 1}),
    datetime(2052, 1, 1): Dual(1, {"v4": 1}),
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.20,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.66,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.93,
    Swap(datetime(2022, 1, 1), 12*30, 12, 12): 2.20,
}
s_cv = SolvedCurve(nodes=nodes, interpolation="log_linear", swaps=list(swaps.keys()), obj_rates=list(swaps.values()))
s_cv.iterate()  

'tolerance reached (gauss_newton) after 5 iterations, func: 5.269421734819802e-25'

### Add a short historical data sample for covariance

In [3]:
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
Q = historical_chgs.cov().to_numpy()
Q

array([[8.66      , 7.3775    , 5.13625   , 2.185     ],
       [7.3775    , 7.3075    , 4.73      , 2.1575    ],
       [5.13625   , 4.73      , 4.89111111, 1.76236111],
       [2.185     , 2.1575    , 1.76236111, 1.86111111]])

### Create a portfolio of swaps replicating the risks in the text

In [4]:
portfolio = Portfolio(objects=[
    Swap(datetime(2022,1,1), 12*2, 12, 12, notional=-1.272e6),
    Swap(datetime(2022,1,1), 12*5, 12, 12, notional=313545),
    Swap(datetime(2022,1,1), 12*10, 12, 12, notional=109931),
    Swap(datetime(2022,1,1), 12*30, 12, 12, notional=27160),
])  

### Utilise all the PCA methods displaying results

In [5]:
lambd, E = portfolio.pca(Q)
print(lambd)
print(E)

[19.58213009  1.52567666  1.05899056  0.5529249 ]
[[-0.64607277  0.40863261 -0.05733184 -0.64212337]
 [-0.5922391   0.25563257  0.18094029  0.74240518]
 [-0.44001486 -0.72630264 -0.5273608   0.0276041 ]
 [-0.19552431 -0.49005699  0.82816922 -0.18907706]]


In [6]:
portfolio.historical_multipliers(Q, historical_chgs)

,0,1,2,3
0,6.153178,-0.406849,-0.768125,-0.152018
1,-2.003498,0.871097,-0.942093,1.564148
2,2.514779,-0.736256,-1.273402,-0.393355
3,0.233179,-0.176537,1.364321,-0.624503
4,-2.718918,2.135376,-0.702667,-0.981917
5,4.570520,0.844200,0.737655,0.077537
6,2.780586,-0.282155,1.346729,0.640112
7,-4.044346,-2.313345,-0.443199,-0.166568
8,-7.485479,0.064470,0.680782,0.036564
9,NaN,NaN,NaN,NaN


In [7]:
portfolio.risk(s_cv)

fwd tolerance reached (gauss_newton) after 2 iterations, func: 2.2162356878892258e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 3.5983594368816996e-26
fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.6081028160543478e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 8.522360181942048e-27
fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.1069779399365686e-25
bck tolerance reached (gauss_newton) after 2 iterations, func: 8.291135189860455e-26
fwd tolerance reached (gauss_newton) after 3 iterations, func: 1.3472659577430203e-26
bck tolerance reached (gauss_newton) after 3 iterations, func: 1.3475617805824782e-26


array([[-249.89298866],
       [ 150.00008767],
       [  99.99986258],
       [  60.00061436]])

In [8]:
portfolio.pca_risk(s_cv, Q)

array([[  16.88013548],
       [-165.80340041],
       [  38.42253791],
       [ 263.23863763]])

In [9]:
portfolio.pca_covar_alloc(s_cv, Q)

array([[ 18.87367781],
       [141.87115991],
       [  5.28821294],
       [129.60154216]])